In [4]:

import functools
import traceback


def log_exceptions(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        func_name = func.__name__
        try:
            result = func(*args, **kwargs)
        except Exception as e:
            print(f"FAILURE: {func_name} failed: {e}")
            traceback.print_exc()
        else:
            print(f"SUCCESS: {func_name} succeeded")
            return result
    return wrapper


def decorate_methods(decorator):
    def decorate(obj):
        if isinstance(obj, type):
            # Decorate class methods
            for name, method in vars(obj).items():
                if callable(method):
                    setattr(obj, name, decorator(method))
            return obj
        else:
            # Decorate standalone functions
            return decorator(obj)
    return decorate


%load_ext autoreload
%autoreload 2

In [15]:
import os 
import sys 

# path to this file's directory and parent directory
current_directory = os.getcwd()
base_directory = os.path.dirname(current_directory)

# parent directory of base_directory
parent_directory = os.path.dirname(base_directory)

# add to path
sys.path.append(base_directory)

# define paths to each app dockerfile location 
processor_path = os.path.join(base_directory, 'quick_batch', 'processor_app')
controller_path = os.path.join(base_directory, 'quick_batch', 'controller_app')
queue_path = os.path.join(base_directory, 'quick_batch', 'queue_app')
config_path = os.path.join(base_directory, 'tests/test_configs', 'test_quick_batch.yaml')
input_path = os.path.join(base_directory, 'tests/test_data', 'input_data')
output_path = os.path.join(base_directory, 'tests/test_data', 'output_data')

print(f'base_directory: {base_directory}')
print(f'processor_path: {processor_path}')
print(f'controller_path: {controller_path}')
print(f'queue_path: {queue_path}')
print(f'config_path: {config_path}')
print(f'input_path: {input_path}')
print(f'output_path: {output_path}')



base_directory: /Users/wattjer/Desktop/quick_batch
processor_path: /Users/wattjer/Desktop/quick_batch/quick_batch/processor_app
controller_path: /Users/wattjer/Desktop/quick_batch/quick_batch/controller_app
queue_path: /Users/wattjer/Desktop/quick_batch/quick_batch/queue_app
config_path: /Users/wattjer/Desktop/quick_batch/tests/test_configs/test_quick_batch.yaml
input_path: /Users/wattjer/Desktop/quick_batch/tests/test_data/input_data
output_path: /Users/wattjer/Desktop/quick_batch/tests/test_data/output_data


# services and swarm

In [1]:
import shutil
import os
import sys
import docker 

In [28]:
@log_exceptions
def create_client():
    return docker.from_env()

@log_exceptions
def create_network(client):
    if 'quick_batch_network' not in [network.name for network in client.networks.list()]:
        client.networks.create('quick_batch_network', driver='overlay')

@log_exceptions
def leave_swarm(client):
    client.swarm.leave(force=True)
        
@log_exceptions
def create_swarm(client):
    client.swarm.init()

@log_exceptions
def remove_service(client, service_name):
    if service_name in [service.name for service in client.services.list()]:
        client.services.get(service_name).remove()

@log_exceptions
def create_queue_service(client):
    # remove queue_app service if it exists
    remove_service(client, 'queue_app')
    
    # Define the mounts for the service containers
    mounts = [
        docker.types.Mount(
            type='bind',
            source=queue_path + '/queue_app',
            target='/queue_app',
            read_only=True
        ),
        docker.types.Mount(
            type='bind',
            source=config_path,
            target='/my_configs/config.yaml',
            read_only=True
        ),
        docker.types.Mount(
            type='bind',
            source=input_path,
            target='/my_data/input',
            read_only=True
        ),
    ]

    # Define the service configuration
    service_config = {
        'image': 'quick_batch_queue_app',
        'name': 'queue_app',
        'mounts': mounts,
        'networks':['quick_batch_network'],
    }

    # Create the service
    service = client.services.create(**service_config)
    return service


In [23]:
# creater docker client
client = docker.from_env()

# remove swarms 
leave_swarm(client)

# create new swarm
create_swarm(client)

# create network
create_network(client)

# create queue service
queue_service = create_queue_service(client)

SUCCESS: leave_swarm succeeded
SUCCESS: create_swarm succeeded
SUCCESS: create_network succeeded


In [29]:
# remove queue_app service if it exists
remove_service(client, 'queue_app')

# Define the mounts for the service containers
mounts = [
    docker.types.Mount(
        type='bind',
        source=queue_path + '/queue_app',
        target='/queue_app',
        read_only=True
    ),
    docker.types.Mount(
        type='bind',
        source=config_path,
        target='/my_configs/config.yaml',
        read_only=True
    ),
    docker.types.Mount(
        type='bind',
        source=input_path,
        target='/my_data/input',
        read_only=True
    ),
]

# Define the service configuration
service_config = {
    'image': 'quick_batch_queue_app',
    'name': 'queue_app',
    'mounts': mounts,
    'networks':['quick_batch_network'],
}

# Create the service
service = client.services.create(**service_config)

In [30]:
queue_service = create_queue_service(client)

SUCCESS: remove_service succeeded
SUCCESS: create_queue_service succeeded


In [18]:

    
# create queue_app service
client.services.create(image='quick_batch_queue_app',
                       name='queue_app',
                       networks=['quick_batch_network'],
                       command='python /queue_app/run.py',
                       mounts={
                                queue_path + '/queue_app':
                                {'bind': '/queue_app', 'mode': 'ro'},
                                config_path:
                                {'bind': '/my_configs/config.yaml', 'mode': 'ro'},
                                input_path:
                                {'bind': '/my_data/input', 'mode': 'ro'},
                        })

APIError: 403 Client Error for http+docker://localhost/v1.41/services/create: Forbidden ("The network quick_batch_network cannot be used with services. Only networks scoped to the swarm can be used, such as those created with the overlay driver.")